# Design 1

In [9]:
import customtkinter as ctk
from tkinter import ttk

# Initialize the main app
ctk.set_appearance_mode("dark")
ctk.set_default_color_theme("blue")

app = ctk.CTk()
app.geometry("900x600")
app.title("AI-Powered Stock Screener")

# Header Frame
header_frame = ctk.CTkFrame(app)
header_frame.pack(fill="x", padx=10, pady=5)

title_label = ctk.CTkLabel(header_frame, text="📈 AI-Powered Stock Screener", font=("Arial", 20, "bold"))
title_label.pack(side="left", padx=10, pady=5)

# Dropdown Filters
filters_frame = ctk.CTkFrame(app)
filters_frame.pack(fill="x", padx=10, pady=5)

sector_label = ctk.CTkLabel(filters_frame, text="Sector:")
sector_label.pack(side="left", padx=5)

sector_dropdown = ctk.CTkComboBox(filters_frame, values=["All", "Tech", "Healthcare", "Finance"])
sector_dropdown.pack(side="left", padx=5)

score_label = ctk.CTkLabel(filters_frame, text="Score Type:")
score_label.pack(side="left", padx=5)

score_dropdown = ctk.CTkComboBox(filters_frame, values=["Overall", "Sentiment", "Technical", "Fundamental"])
score_dropdown.pack(side="left", padx=5)

# Table Frame
table_frame = ctk.CTkFrame(app)
table_frame.pack(fill="both", expand=True, padx=10, pady=5)

columns = ("Rank", "Ticker", "Overall Score", "Sentiment", "Technical", "Fundamental", "Trend")
tree = ttk.Treeview(table_frame, columns=columns, show="headings", height=8)

for col in columns:
    tree.heading(col, text=col)
    tree.column(col, width=100)

# Dummy Data
dummy_data = [
    (1, "AAPL", 92, 85, 88, 78, "📈 Bullish"),
    (2, "MSFT", 89, 90, 80, 85, "📉 Bearish"),
    (3, "GOOGL", 85, 80, 75, 90, "📈 Bullish"),
]

for row in dummy_data:
    tree.insert("", "end", values=row)

tree.pack(fill="both", expand=True)

# Stock Detail Frame
detail_frame = ctk.CTkFrame(app)
detail_frame.pack(fill="x", padx=10, pady=5)

detail_label = ctk.CTkLabel(detail_frame, text="Stock Details", font=("Arial", 16, "bold"))
detail_label.pack(pady=5)

# Graph Placeholder
graph_frame = ctk.CTkFrame(app, height=150)
graph_frame.pack(fill="x", padx=10, pady=5)
graph_label = ctk.CTkLabel(graph_frame, text="📊 Sentiment & Technical Chart Placeholder")
graph_label.pack(pady=10)

# Run the App
app.mainloop()


In [10]:
import customtkinter as ctk
import tkinter as tk  # required for Canvas & integration with matplotlib
from tkinter import ttk  # for Treeview widget
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class TradingApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("AI Trading Platform")
        self.geometry("1200x800")

        # Create a Tabview widget from customtkinter
        self.notebook = ctk.CTkTabview(self, width=1100, height=700)
        self.notebook.pack(padx=20, pady=20, fill="both", expand=True)
        
        # Add Tabs
        self.notebook.add("Dashboard")
        self.notebook.add("Stock Analysis")
        self.notebook.add("Backtesting")
        self.notebook.add("Settings")

        # Build each tab's UI
        self.create_dashboard_content(self.notebook.tab("Dashboard"))
        self.create_analysis_content(self.notebook.tab("Stock Analysis"))
        self.create_backtesting_content(self.notebook.tab("Backtesting"))
        self.create_settings_content(self.notebook.tab("Settings"))

        # Create a status bar at the bottom
        self.status_bar = ctk.CTkLabel(self, text="Ready", anchor="w")
        self.status_bar.pack(padx=20, pady=(0,20), fill="x")

    def create_dashboard_content(self, tab):
        # Metrics Frame
        metrics_frame = ctk.CTkFrame(tab)
        metrics_frame.pack(pady=10, padx=10)
        
        ctk.CTkLabel(metrics_frame, text="Total Portfolio Value:").grid(row=0, column=0, padx=5, pady=5)
        self.portfolio_value_label = ctk.CTkLabel(metrics_frame, text="$0.00")
        self.portfolio_value_label.grid(row=0, column=1, padx=5, pady=5)

        ctk.CTkLabel(metrics_frame, text="Daily Change:").grid(row=1, column=0, padx=5, pady=5)
        self.daily_change_label = ctk.CTkLabel(metrics_frame, text="$0.00")
        self.daily_change_label.grid(row=1, column=1, padx=5, pady=5)

        ctk.CTkLabel(metrics_frame, text="Overall Return:").grid(row=2, column=0, padx=5, pady=5)
        self.overall_return_label = ctk.CTkLabel(metrics_frame, text="0.00%")
        self.overall_return_label.grid(row=2, column=1, padx=5, pady=5)

        # Top Performing Stocks
        top_stocks_frame = ctk.CTkFrame(tab)
        top_stocks_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(top_stocks_frame, text="Top Performing Stocks:").pack(pady=5)
        # Using tk.Listbox for now as CustomTkinter does not have a native Listbox
        self.top_stocks_listbox = tk.Listbox(top_stocks_frame, width=50, height=10)
        self.top_stocks_listbox.pack(pady=5)

        # Recent News
        news_frame = ctk.CTkFrame(tab)
        news_frame.pack(pady=10, padx=10, fill="both", expand=True)
        ctk.CTkLabel(news_frame, text="Recent News:").pack(pady=5)
        # Using tk.Text for news content
        self.news_text = tk.Text(news_frame, width=80, height=10, wrap=tk.WORD)
        self.news_text.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, pady=5, padx=5)
        news_scrollbar = ctk.CTkScrollbar(news_frame, command=self.news_text.yview)
        news_scrollbar.pack(side=tk.RIGHT, fill=tk.Y, pady=5)
        self.news_text.configure(yscrollcommand=news_scrollbar.set)

    def create_analysis_content(self, tab):
        # Symbol Input
        symbol_frame = ctk.CTkFrame(tab)
        symbol_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(symbol_frame, text="Stock Symbol:").grid(row=0, column=0, padx=5, pady=5)
        self.symbol_entry = ctk.CTkEntry(symbol_frame)
        self.symbol_entry.grid(row=0, column=1, padx=5, pady=5)
        analyze_button = ctk.CTkButton(symbol_frame, text="Analyze", command=self.analyze_stock)
        analyze_button.grid(row=0, column=2, padx=5, pady=5)

        # Technical Indicators Chart area
        chart_frame = ctk.CTkFrame(tab)
        chart_frame.pack(pady=10, padx=10)
        ctk.CTkLabel(chart_frame, text="Technical Indicators Chart:").pack(pady=5)
        # Using tk.Canvas for embedding matplotlib chart
        self.chart_canvas = tk.Canvas(chart_frame, width=800, height=300, bg="white")
        self.chart_canvas.pack(pady=5)

        # Fundamental Data Table
        fundamental_frame = ctk.CTkFrame(tab)
        fundamental_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(fundamental_frame, text="Fundamental Data:").pack(pady=5)
        # Using ttk Treeview for tabular data
        self.fundamental_tree = ttk.Treeview(fundamental_frame, columns=("Metric", "Value"), show="headings")
        self.fundamental_tree.heading("Metric", text="Metric")
        self.fundamental_tree.heading("Value", text="Value")
        self.fundamental_tree.pack(pady=5)

        # Sentiment Analysis Results
        sentiment_frame = ctk.CTkFrame(tab)
        sentiment_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(sentiment_frame, text="Sentiment Analysis:").pack(pady=5)
        self.sentiment_label = ctk.CTkLabel(sentiment_frame, text="Sentiment Score: N/A")
        self.sentiment_label.pack(pady=5)
        self.sentiment_text = tk.Text(sentiment_frame, width=80, height=5, wrap=tk.WORD)
        self.sentiment_text.pack(pady=5)

    def create_backtesting_content(self, tab):
        # Strategy Configuration
        strategy_frame = ctk.CTkFrame(tab)
        strategy_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(strategy_frame, text="Select Strategy:").grid(row=0, column=0, padx=5, pady=5)
        self.strategy_combo = ctk.CTkComboBox(strategy_frame, values=["Strategy 1", "Strategy 2"])
        self.strategy_combo.grid(row=0, column=1, padx=5, pady=5)
        ctk.CTkLabel(strategy_frame, text="Initial Capital:").grid(row=1, column=0, padx=5, pady=5)
        self.initial_capital_entry = ctk.CTkEntry(strategy_frame)
        self.initial_capital_entry.grid(row=1, column=1, padx=5, pady=5)

        # Backtesting Parameters
        params_frame = ctk.CTkFrame(tab)
        params_frame.pack(pady=10, padx=10, fill="x")
        ctk.CTkLabel(params_frame, text="Start Date:").grid(row=0, column=0, padx=5, pady=5)
        self.start_date_entry = ctk.CTkEntry(params_frame)
        self.start_date_entry.grid(row=0, column=1, padx=5, pady=5)
        ctk.CTkLabel(params_frame, text="End Date:").grid(row=1, column=0, padx=5, pady=5)
        self.end_date_entry = ctk.CTkEntry(params_frame)
        self.end_date_entry.grid(row=1, column=1, padx=5, pady=5)
        run_button = ctk.CTkButton(params_frame, text="Run Backtest", command=self.run_backtest)
        run_button.grid(row=2, column=0, columnspan=2, pady=5)

        # Performance Chart Area
        charts_frame = ctk.CTkFrame(tab)
        charts_frame.pack(pady=10, padx=10)
        ctk.CTkLabel(charts_frame, text="Performance Chart:").pack(pady=5)
        self.performance_canvas = tk.Canvas(charts_frame, width=800, height=300, bg="white")
        self.performance_canvas.pack(pady=5)

    def create_settings_content(self, tab):
        # Basic Settings content
        label = ctk.CTkLabel(tab, text="Settings Content Here")
        label.pack(padx=50, pady=50)

    def analyze_stock(self):
        symbol = self.symbol_entry.get()
        try:
            # Fetch historical data using yfinance
            stock_data = yf.Ticker(symbol)
            history = stock_data.history(period="1y")
            
            # Plot stock closing price
            fig, ax = plt.subplots(figsize=(8, 3))
            ax.plot(history['Close'])
            ax.set_title(f"{symbol} Stock Price")
            ax.set_xlabel("Date")
            ax.set_ylabel("Price (USD)")
            fig.tight_layout()

            # Embed matplotlib chart in tkinter Canvas
            canvas = FigureCanvasTkAgg(fig, master=self.chart_canvas)
            canvas_widget = canvas.get_tk_widget()
            canvas_widget.pack(fill=tk.BOTH, expand=True)
            canvas.draw()

            # Populate Fundamental Data
            info = stock_data.info
            self.populate_fundamental_data(info)

            # Fetch news and perform Sentiment Analysis using TextBlob
            news = stock_data.news
            if news:
                combined_news = " ".join([article.get('title', '') + " " + article.get('summary', '') for article in news])
                sentiment = TextBlob(combined_news).sentiment.polarity
                self.sentiment_label.configure(text=f"Sentiment Score: {sentiment:.2f}")
                self.sentiment_text.delete("1.0", tk.END)
                self.sentiment_text.insert(tk.END, combined_news)
            else:
                self.sentiment_label.configure(text="No News Available")
                self.sentiment_text.delete("1.0", tk.END)
                self.sentiment_text.insert(tk.END, "No News Available")
        except Exception as e:
            print(e)
            self.status_bar.configure(text=f"Error: Could not analyze {symbol}")

    def populate_fundamental_data(self, info):
        # Clear previous fundamental data
        for item in self.fundamental_tree.get_children():
            self.fundamental_tree.delete(item)

        data = {
            "Market Cap": info.get("marketCap", "N/A"),
            "Forward PE": info.get("forwardPE", "N/A"),
            "Dividend Yield": info.get("dividendYield", "N/A"),
            "52 Week High": info.get("fiftyTwoWeekHigh", "N/A"),
            "52 Week Low": info.get("fiftyTwoWeekLow", "N/A")
        }
        for metric, value in data.items():
            self.fundamental_tree.insert("", tk.END, values=(metric, value))

    def run_backtest(self):
        # Placeholder for your backtesting algorithm
        self.status_bar.configure(text="Running Backtest...")
        # Implement your backtesting logic here...
        self.status_bar.configure(text="Backtest Complete")

if __name__ == "__main__":
    ctk.set_appearance_mode("System")  # Options: "System", "Dark", "Light"
    ctk.set_default_color_theme("blue")  # Options: "blue", "green", "dark-blue"
    app = TradingApp()
    app.mainloop()

In [6]:
import customtkinter as ctk
import tkinter as tk
from tkinter import ttk
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from PIL import Image, ImageTk
import os

class ModernTradingApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Modern Trading Platform")
        self.geometry("1400x800")
        
        # Configure grid layout
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(1, weight=1)

        # Create sidebar frame with widgets
        self.sidebar_frame = ctk.CTkFrame(self, width=140, corner_radius=0)
        self.sidebar_frame.grid(row=0, column=0, rowspan=4, sticky="nsew")
        self.sidebar_frame.grid_rowconfigure(4, weight=1)

        self.logo_label = ctk.CTkLabel(self.sidebar_frame, text="Trading Platform", font=ctk.CTkFont(size=20, weight="bold"))
        self.logo_label.grid(row=0, column=0, padx=20, pady=(20, 10))

        # Sidebar buttons
        self.dashboard_button = ctk.CTkButton(self.sidebar_frame, text="Dashboard", command=self.show_dashboard)
        self.dashboard_button.grid(row=1, column=0, padx=20, pady=10)
        
        self.analysis_button = ctk.CTkButton(self.sidebar_frame, text="Stock Analysis", command=self.show_analysis)
        self.analysis_button.grid(row=2, column=0, padx=20, pady=10)
        
        self.backtest_button = ctk.CTkButton(self.sidebar_frame, text="Backtesting", command=self.show_backtest)
        self.backtest_button.grid(row=3, column=0, padx=20, pady=10)

        # Create main content area
        self.main_frame = ctk.CTkFrame(self)
        self.main_frame.grid(row=0, column=1, padx=20, pady=20, sticky="nsew")

        # Create search frame at top
        self.search_frame = ctk.CTkFrame(self.main_frame)
        self.search_frame.pack(fill="x", padx=5, pady=5)
        
        self.search_entry = ctk.CTkEntry(self.search_frame, placeholder_text="Search stocks...")
        self.search_entry.pack(side="left", padx=5, pady=5, fill="x", expand=True)
        
        self.search_button = ctk.CTkButton(self.search_frame, text="Search", width=100)
        self.search_button.pack(side="right", padx=5, pady=5)

        # Create split view
        self.paned_window = ttk.PanedWindow(self.main_frame, orient="horizontal")
        self.paned_window.pack(fill="both", expand=True, padx=5, pady=5)

        # Left panel (Filter + Results)
        self.left_panel = ctk.CTkFrame(self.paned_window)
        self.paned_window.add(self.left_panel, weight=1)

        # Filter section
        self.filter_frame = ctk.CTkFrame(self.left_panel)
        self.filter_frame.pack(fill="x", padx=5, pady=5)
        
        # Add filter controls
        self.create_filter_controls()

        # Results section (using cards)
        self.results_frame = ctk.CTkFrame(self.left_panel)
        self.results_frame.pack(fill="both", expand=True, padx=5, pady=5)
        
        # Right panel (Details view)
        self.right_panel = ctk.CTkFrame(self.paned_window)
        self.paned_window.add(self.right_panel, weight=2)

        # Initialize with dashboard view
        self.show_dashboard()

    def create_filter_controls(self):
        # Price Range
        price_frame = ctk.CTkFrame(self.filter_frame)
        price_frame.pack(fill="x", padx=5, pady=2)
        
        ctk.CTkLabel(price_frame, text="Price Range:").pack(side="left", padx=5)
        self.price_min = ctk.CTkEntry(price_frame, width=80, placeholder_text="Min")
        self.price_min.pack(side="left", padx=2)
        self.price_max = ctk.CTkEntry(price_frame, width=80, placeholder_text="Max")
        self.price_max.pack(side="left", padx=2)

        # Market Cap
        cap_frame = ctk.CTkFrame(self.filter_frame)
        cap_frame.pack(fill="x", padx=5, pady=2)
        
        ctk.CTkLabel(cap_frame, text="Market Cap:").pack(side="left", padx=5)
        self.market_cap = ctk.CTkComboBox(cap_frame, values=["Any", "Micro", "Small", "Mid", "Large", "Mega"])
        self.market_cap.pack(side="left", padx=2)

        # Apply Filters button
        self.apply_filters = ctk.CTkButton(self.filter_frame, text="Apply Filters", command=self.update_results)
        self.apply_filters.pack(pady=5)

    def create_stock_card(self, symbol, price, change):
        card = ctk.CTkFrame(self.results_frame)
        card.pack(fill="x", padx=5, pady=2)
        
        ctk.CTkLabel(card, text=symbol, font=ctk.CTkFont(size=16, weight="bold")).pack(side="left", padx=10)
        ctk.CTkLabel(card, text=f"${price}").pack(side="left", padx=10)
        
        change_color = "green" if change >= 0 else "red"
        ctk.CTkLabel(card, text=f"{change:+.2f}%", text_color=change_color).pack(side="left", padx=10)
        
        ctk.CTkButton(card, text="View Details", width=100, command=lambda: self.show_stock_details(symbol)).pack(side="right", padx=10)

    def show_stock_details(self, symbol):
        # Clear previous content
        for widget in self.right_panel.winfo_children():
            widget.destroy()

        # Create details view
        ctk.CTkLabel(self.right_panel, text=symbol, font=ctk.CTkFont(size=20, weight="bold")).pack(pady=10)
        
        # Chart area
        chart_frame = ctk.CTkFrame(self.right_panel)
        chart_frame.pack(fill="x", padx=10, pady=5)
        
        # Tabs for different data views
        tabview = ctk.CTkTabview(self.right_panel)
        tabview.pack(fill="both", expand=True, padx=10, pady=5)
        
        tabview.add("Overview")
        tabview.add("Financials")
        tabview.add("News")

    def show_dashboard(self):
        self.clear_main_content()
        # Add dashboard content here

    def show_analysis(self):
        self.clear_main_content()
        # Add analysis content here

    def show_backtest(self):
        self.clear_main_content()
        # Add backtesting content here

    def clear_main_content(self):
        for widget in self.main_frame.winfo_children():
            widget.destroy()

    def update_results(self):
        # Clear previous results
        for widget in self.results_frame.winfo_children():
            widget.destroy()
            
        # Add sample stock cards
        self.create_stock_card("AAPL", 150.25, 2.5)
        self.create_stock_card("MSFT", 285.30, -1.2)
        self.create_stock_card("GOOGL", 2750.15, 0.8)

if __name__ == "__main__":
    app = ModernTradingApp()
    app.mainloop()

In [7]:
import customtkinter as ctk
import tkinter as tk
from tkinter import ttk
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd

class StockScreenerApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Advanced Stock Screener")
        self.geometry("1400x800")
        
        # Set dark theme
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
        
        # Configure main grid
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        # Create main sections
        self.create_filter_sidebar()
        self.create_main_content()
        self.create_detail_panel()
        
    def create_filter_sidebar(self):
        # Left sidebar for filters
        self.filter_panel = ctk.CTkFrame(self, width=250)
        self.filter_panel.grid(row=0, column=0, padx=10, pady=10, sticky="nsew")
        
        # Filter header
        ctk.CTkLabel(self.filter_panel, text="Screening Criteria", 
                    font=("Roboto", 16, "bold")).pack(pady=10)
        
        # Price Range
        self.create_range_filter("Price ($)", 0, 1000)
        
        # Market Cap Filter
        self.create_dropdown_filter("Market Cap", 
            ["Any", "Mega (>200B)", "Large (10-200B)", "Mid (2-10B)", "Small (<2B)"])
        
        # Sector Filter
        self.create_dropdown_filter("Sector",
            ["Any", "Technology", "Healthcare", "Financial", "Consumer", "Energy"])
        
        # Technical Indicators
        self.create_range_filter("RSI", 0, 100)
        self.create_range_filter("P/E Ratio", 0, 100)
        
        # Apply Filters Button
        ctk.CTkButton(self.filter_panel, text="Apply Filters", 
                     command=self.apply_filters).pack(pady=20)
        
    def create_main_content(self):
        # Main content area
        self.main_panel = ctk.CTkFrame(self)
        self.main_panel.grid(row=0, column=1, padx=10, pady=10, sticky="nsew")
        
        # Search bar at top
        search_frame = ctk.CTkFrame(self.main_panel)
        search_frame.pack(fill="x", padx=5, pady=5)
        
        self.search_entry = ctk.CTkEntry(search_frame, placeholder_text="Search stocks...")
        self.search_entry.pack(side="left", fill="x", expand=True, padx=5)
        
        # Results area
        self.create_results_table()
        
    def create_detail_panel(self):
        # Right panel for stock details
        self.detail_panel = ctk.CTkFrame(self, width=400)
        self.detail_panel.grid(row=0, column=2, padx=10, pady=10, sticky="nsew")
        
        # Tabbed interface for details
        self.detail_tabs = ctk.CTkTabview(self.detail_panel)
        self.detail_tabs.pack(fill="both", expand=True)
        
        # Add detail tabs
        self.detail_tabs.add("Overview")
        self.detail_tabs.add("Charts")
        self.detail_tabs.add("Financials")
        
    def create_range_filter(self, name, min_val, max_val):
        frame = ctk.CTkFrame(self.filter_panel)
        frame.pack(fill="x", padx=10, pady=5)
        
        ctk.CTkLabel(frame, text=name).pack()
        
        range_frame = ctk.CTkFrame(frame)
        range_frame.pack(fill="x", pady=2)
        
        min_entry = ctk.CTkEntry(range_frame, placeholder_text="Min")
        min_entry.pack(side="left", padx=2, expand=True)
        
        max_entry = ctk.CTkEntry(range_frame, placeholder_text="Max")
        max_entry.pack(side="left", padx=2, expand=True)
        
        return min_entry, max_entry
        
    def create_dropdown_filter(self, name, options):
        frame = ctk.CTkFrame(self.filter_panel)
        frame.pack(fill="x", padx=10, pady=5)
        
        ctk.CTkLabel(frame, text=name).pack()
        
        dropdown = ctk.CTkComboBox(frame, values=options)
        dropdown.pack(fill="x", pady=2)
        
        return dropdown
        
    def create_results_table(self):
        # Custom styled treeview
        style = ttk.Style()
        style.configure("Custom.Treeview", background="#2b2b2b", 
                      fieldbackground="#2b2b2b", foreground="white")
        
        columns = ("Symbol", "Company", "Price", "Change", "Volume", "Market Cap")
        self.results_tree = ttk.Treeview(self.main_panel, columns=columns, 
                                       show="headings", style="Custom.Treeview")
        
        # Configure columns
        for col in columns:
            self.results_tree.heading(col, text=col)
            self.results_tree.column(col, width=100)
            
        self.results_tree.pack(fill="both", expand=True, padx=5, pady=5)
        
        # Add scrollbar
        scrollbar = ctk.CTkScrollbar(self.main_panel, 
                                   command=self.results_tree.yview)
        scrollbar.pack(side="right", fill="y")
        
        self.results_tree.configure(yscrollcommand=scrollbar.set)
        
    def apply_filters(self):
        # Implement filter logic here
        pass

if __name__ == "__main__":
    app = StockScreenerApp()
    app.mainloop()

In [8]:
import customtkinter as ctk
import tkinter as tk
from tkinter import ttk
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import pandas as pd

class DashboardApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Stock Trading Dashboard")
        self.geometry("1400x800")
        
        # Configure theme and colors
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
        
        # Main container
        self.container = ctk.CTkFrame(self)
        self.container.pack(fill="both", expand=True, padx=20, pady=20)
        
        # Create layout
        self.create_header()
        self.create_dashboard()
        
    def create_header(self):
        header = ctk.CTkFrame(self.container)
        header.pack(fill="x", pady=(0, 20))
        
        # Logo and title
        title = ctk.CTkLabel(header, text="Trading Dashboard", 
                           font=("Helvetica", 24, "bold"))
        title.pack(side="left", padx=20)
        
        # Quick actions
        actions = ctk.CTkFrame(header)
        actions.pack(side="right", padx=20)
        
        ctk.CTkButton(actions, text="New Trade", 
                     command=self.new_trade).pack(side="left", padx=5)
        ctk.CTkButton(actions, text="Settings", 
                     command=self.show_settings).pack(side="left", padx=5)
        
    def create_dashboard(self):
        # Create grid layout
        self.dashboard = ctk.CTkFrame(self.container)
        self.dashboard.pack(fill="both", expand=True)
        self.dashboard.grid_columnconfigure((0,1,2), weight=1)
        
        # Market Overview Card
        self.create_market_card()
        
        # Watchlist Card
        self.create_watchlist_card()
        
        # Portfolio Card
        self.create_portfolio_card()
        
        # News Feed Card
        self.create_news_card()
        
        # Trading Activity Card
        self.create_activity_card()
        
        # Performance Metrics Card
        self.create_metrics_card()
        
    def create_market_card(self):
        card = self.create_card("Market Overview", 0, 0)
        
        # Market indices
        indices = ["S&P 500", "NASDAQ", "DOW"]
        values = ["4,532.12", "14,234.45", "35,124.67"]
        changes = ["+1.2%", "-0.5%", "+0.8%"]
        
        for i, (index, value, change) in enumerate(zip(indices, values, changes)):
            frame = ctk.CTkFrame(card)
            frame.pack(fill="x", pady=2)
            ctk.CTkLabel(frame, text=index).pack(side="left", padx=5)
            ctk.CTkLabel(frame, text=value).pack(side="left", padx=5)
            color = "green" if "+" in change else "red"
            ctk.CTkLabel(frame, text=change, text_color=color).pack(side="right", padx=5)
            
    def create_watchlist_card(self):
        card = self.create_card("Watchlist", 0, 1, rowspan=2)
        
        # Search bar
        search_frame = ctk.CTkFrame(card)
        search_frame.pack(fill="x", pady=5)
        ctk.CTkEntry(search_frame, placeholder_text="Add symbol...").pack(side="left", 
                                                                        fill="x", 
                                                                        expand=True, 
                                                                        padx=5)
        ctk.CTkButton(search_frame, text="+", width=30).pack(side="right", padx=5)
        
        # Watchlist table
        columns = ("Symbol", "Price", "Change")
        self.watchlist = ttk.Treeview(card, columns=columns, show="headings", height=10)
        for col in columns:
            self.watchlist.heading(col, text=col)
        self.watchlist.pack(fill="both", expand=True, pady=5)
        
    def create_portfolio_card(self):
        card = self.create_card("Portfolio Summary", 0, 2)
        
        # Portfolio value
        value_frame = ctk.CTkFrame(card)
        value_frame.pack(fill="x", pady=5)
        ctk.CTkLabel(value_frame, text="Total Value:").pack(side="left", padx=5)
        ctk.CTkLabel(value_frame, text="$125,432.67", 
                    font=("Helvetica", 16, "bold")).pack(side="right", padx=5)
        
        # Daily P/L
        pl_frame = ctk.CTkFrame(card)
        pl_frame.pack(fill="x", pady=5)
        ctk.CTkLabel(pl_frame, text="Today's P/L:").pack(side="left", padx=5)
        ctk.CTkLabel(pl_frame, text="+$1,234.56", 
                    text_color="green").pack(side="right", padx=5)
        
    def create_news_card(self):
        card = self.create_card("Market News", 1, 0)
        
        # News feed
        self.news_text = tk.Text(card, height=8, bg="#2b2b2b", fg="white", 
                                wrap=tk.WORD)
        self.news_text.pack(fill="both", expand=True, pady=5)
        
    def create_activity_card(self):
        card = self.create_card("Recent Activity", 1, 2)
        
        # Activity list
        activities = ttk.Treeview(card, columns=("Time", "Action"), 
                                show="headings", height=8)
        activities.heading("Time", text="Time")
        activities.heading("Action", text="Action")
        activities.pack(fill="both", expand=True, pady=5)
        
    def create_metrics_card(self):
        card = self.create_card("Performance Metrics", 2, 0, columnspan=3)
        
        # Metrics grid
        metrics_frame = ctk.CTkFrame(card)
        metrics_frame.pack(fill="both", expand=True, pady=5)
        
        metrics = [
            ("Win Rate", "65%"),
            ("Avg Return", "2.3%"),
            ("Sharpe Ratio", "1.8"),
            ("Max Drawdown", "-12%")
        ]
        
        for i, (label, value) in enumerate(metrics):
            frame = ctk.CTkFrame(metrics_frame)
            frame.grid(row=0, column=i, padx=10, pady=5, sticky="nsew")
            ctk.CTkLabel(frame, text=label).pack()
            ctk.CTkLabel(frame, text=value, 
                        font=("Helvetica", 16, "bold")).pack()
            
    def create_card(self, title, row, column, rowspan=1, columnspan=1):
        card = ctk.CTkFrame(self.dashboard)
        card.grid(row=row, column=column, rowspan=rowspan, columnspan=columnspan,
                 padx=10, pady=10, sticky="nsew")
        
        ctk.CTkLabel(card, text=title, 
                    font=("Helvetica", 14, "bold")).pack(pady=10)
        return card
        
    def new_trade(self):
        # Implement new trade functionality
        pass
        
    def show_settings(self):
        # Implement settings dialog
        pass

if __name__ == "__main__":
    app = DashboardApp()
    app.mainloop()

In [11]:
import customtkinter as ctk
from tkinter import ttk

class AdvancedScreenerApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Advanced Stock Screener")
        self.geometry("1200x800")
        
        # Configure theme
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
        
        # Create main sections
        self.create_header()
        self.create_criteria_tabs()
        self.create_results_section()
        
    def create_header(self):
        header = ctk.CTkFrame(self)
        header.pack(fill="x", padx=20, pady=10)
        
        title = ctk.CTkLabel(header, text="Advanced Stock Screener", 
                           font=("Helvetica", 24, "bold"))
        title.pack(side="left", padx=20)
        
        scan_button = ctk.CTkButton(header, text="Run Scan", 
                                  command=self.run_scan,
                                  width=120)
        scan_button.pack(side="right", padx=20)
        
    def create_criteria_tabs(self):
        # Tabview for different criteria
        self.tabs = ctk.CTkTabview(self)
        self.tabs.pack(fill="x", padx=20, pady=10)
        
        # Create tabs
        self.tabs.add("Sentiment")
        self.tabs.add("Technical")
        self.tabs.add("Fundamental")
        self.tabs.add("Overall")
        
        # Populate each tab
        self.create_sentiment_criteria(self.tabs.tab("Sentiment"))
        self.create_technical_criteria(self.tabs.tab("Technical"))
        self.create_fundamental_criteria(self.tabs.tab("Fundamental"))
        self.create_overall_criteria(self.tabs.tab("Overall"))
        
    def create_sentiment_criteria(self, tab):
        criteria_frame = ctk.CTkFrame(tab)
        criteria_frame.pack(fill="x", padx=10, pady=5)
        
        # News Sentiment
        self.create_dropdown("News Sentiment", [
            "Very Bullish (>80%)",
            "Bullish (60-80%)",
            "Neutral (40-60%)",
            "Bearish (20-40%)",
            "Very Bearish (<20%)"
        ], criteria_frame)
        
        # Social Media Impact
        self.create_dropdown("Social Media Impact", [
            "High Positive Momentum",
            "Medium Positive Momentum",
            "Low Impact",
            "Medium Negative Momentum",
            "High Negative Momentum"
        ], criteria_frame)
        
        # Analyst Coverage
        self.create_dropdown("Analyst Coverage", [
            "Strong Buy",
            "Buy",
            "Hold",
            "Sell",
            "Strong Sell"
        ], criteria_frame)
        
    def create_technical_criteria(self, tab):
        criteria_frame = ctk.CTkFrame(tab)
        criteria_frame.pack(fill="x", padx=10, pady=5)
        
        # Moving Averages
        self.create_dropdown("Moving Averages", [
            "Above 200 MA",
            "Above 50 MA",
            "Above 20 MA",
            "Below 20 MA",
            "Below 50 MA",
            "Below 200 MA"
        ], criteria_frame)
        
        # RSI Conditions
        self.create_dropdown("RSI (14)", [
            "Overbought (>70)",
            "Bullish (60-70)",
            "Neutral (40-60)",
            "Bearish (30-40)",
            "Oversold (<30)"
        ], criteria_frame)
        
        # Volume
        self.create_dropdown("Volume", [
            "Heavy (>2x Avg)",
            "Above Average",
            "Normal",
            "Below Average",
            "Light (<0.5x Avg)"
        ], criteria_frame)
        
    def create_fundamental_criteria(self, tab):
        criteria_frame = ctk.CTkFrame(tab)
        criteria_frame.pack(fill="x", padx=10, pady=5)
        
        # P/E Ratio
        self.create_dropdown("P/E Ratio", [
            "Very Low (<10)",
            "Low (10-15)",
            "Medium (15-25)",
            "High (25-50)",
            "Very High (>50)"
        ], criteria_frame)
        
        # Market Cap
        self.create_dropdown("Market Cap", [
            "Mega (>$200B)",
            "Large ($10B-$200B)",
            "Mid ($2B-$10B)",
            "Small ($300M-$2B)",
            "Micro (<$300M)"
        ], criteria_frame)
        
        # Dividend Yield
        self.create_dropdown("Dividend Yield", [
            "High (>5%)",
            "Above Average (3-5%)",
            "Average (1-3%)",
            "Low (<1%)",
            "No Dividend"
        ], criteria_frame)
        
    def create_overall_criteria(self, tab):
        criteria_frame = ctk.CTkFrame(tab)
        criteria_frame.pack(fill="x", padx=10, pady=5)
        
        # Combined Score
        self.create_dropdown("Combined Score", [
            "Extremely Bullish (90-100)",
            "Very Bullish (80-90)",
            "Bullish (70-80)",
            "Slightly Bullish (60-70)",
            "Neutral (40-60)",
            "Slightly Bearish (30-40)",
            "Bearish (20-30)",
            "Very Bearish (10-20)",
            "Extremely Bearish (0-10)"
        ], criteria_frame)
        
    def create_dropdown(self, label, options, parent):
        frame = ctk.CTkFrame(parent)
        frame.pack(fill="x", padx=5, pady=5)
        
        ctk.CTkLabel(frame, text=label).pack(side="left", padx=10)
        dropdown = ctk.CTkComboBox(frame, values=options, width=200)
        dropdown.pack(side="right", padx=10)
        return dropdown
        
    def create_results_section(self):
        results_frame = ctk.CTkFrame(self)
        results_frame.pack(fill="both", expand=True, padx=20, pady=10)
        
        # Results table
        columns = ("Rank", "Ticker", "Company", "Score", "Sentiment", "Technical", "Fundamental")
        self.results_tree = ttk.Treeview(results_frame, columns=columns, show="headings")
        
        for col in columns:
            self.results_tree.heading(col, text=col)
            self.results_tree.column(col, width=100)
            
        self.results_tree.pack(fill="both", expand=True, padx=5, pady=5)
        
    def run_scan(self):
        # Implement scan logic here
        pass

if __name__ == "__main__":
    app = AdvancedScreenerApp()
    app.mainloop()

In [14]:
import customtkinter as ctk
from tkinter import ttk
import threading

class SimpleScreenerApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Stock Screener - Top Picks")
        self.geometry("1200x800")
        
        # Configure theme
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
        
        # Sample symbols (for testing)
        self.symbols = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'JPM', 'V']
        
        # Create main layout
        self.create_gui()
        
    def create_gui(self):
        # Create notebook for tabs
        self.notebook = ttk.Notebook(self)
        self.notebook.pack(fill="both", expand=True, padx=10, pady=5)
        
        # Create tabs with different scoring criteria
        self.tabs = {
            "Fundamental": {
                "criteria": ["P/E Ratio", "Market Cap", "Dividend Yield", "ROE"],
                "tab": None
            },
            "Technical": {
                "criteria": ["RSI", "Moving Averages", "Volume", "Price Action"],
                "tab": None
            },
            "Sentiment": {
                "criteria": ["News Sentiment", "Social Media", "Analyst Ratings"],
                "tab": None
            },
            "Overall": {
                "criteria": ["Combined Score"],
                "tab": None
            }
        }
        
        # Create each tab
        for category, content in self.tabs.items():
            content["tab"] = self.create_tab(category, content["criteria"])
    
    def create_tab(self, name, criteria):
        tab = ctk.CTkFrame(self.notebook)
        self.notebook.add(tab, text=name)
        
        # Criteria selection
        criteria_frame = ctk.CTkFrame(tab)
        criteria_frame.pack(fill="x", padx=10, pady=5)
        
        ctk.CTkLabel(criteria_frame, text="Select Metric:").pack(side="left", padx=5)
        metric_var = ctk.StringVar(value=criteria[0])
        metric_dropdown = ctk.CTkComboBox(criteria_frame, 
                                        values=criteria,
                                        variable=metric_var)
        metric_dropdown.pack(side="left", padx=5)
        
        scan_button = ctk.CTkButton(criteria_frame, 
                                  text="Find Top 10",
                                  command=lambda: self.run_scan(name))
        scan_button.pack(side="right", padx=10)
        
        # Progress bar
        progress = ctk.CTkProgressBar(tab)
        progress.pack(fill="x", padx=10, pady=5)
        progress.set(0)
        
        # Results table
        columns = ("Rank", "Symbol", "Company", f"{name} Score", "Details")
        tree = ttk.Treeview(tab, columns=columns, show="headings", height=10)
        
        # Configure column widths and headings
        widths = {
            "Rank": 50,
            "Symbol": 100,
            "Company": 300,
            f"{name} Score": 100,
            "Details": 300
        }
        
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=widths.get(col, 150))
        
        tree.pack(fill="both", expand=True, padx=10, pady=5)
        
        # Add scrollbar
        scrollbar = ttk.Scrollbar(tab, orient="vertical", command=tree.yview)
        scrollbar.pack(side="right", fill="y")
        tree.configure(yscrollcommand=scrollbar.set)
        
        return {
            "frame": tab,
            "metric": metric_var,
            "progress": progress,
            "tree": tree
        }
    
    def run_scan(self, category):
        def scan_thread():
            # Get tab components
            tab = self.tabs[category]["tab"]
            tree = tab["tree"]
            progress = tab["progress"]
            selected_metric = tab["metric"].get()
            
            # Clear existing results
            tree.delete(*tree.get_children())
            
            # Simulate scanning process
            total_steps = len(self.symbols)
            for i in range(total_steps):
                # Update progress
                progress.set((i + 1) / total_steps)
                self.update_idletasks()
                
                # Simulate processing delay
                import time
                time.sleep(0.2)
            
            # Add dummy results for testing
            dummy_results = [
                (1, "AAPL", "Apple Inc.", "95.5", "Strong Performance"),
                (2, "MSFT", "Microsoft Corp.", "92.3", "Above Average"),
                (3, "GOOGL", "Alphabet Inc.", "88.7", "Positive Trend"),
                (4, "AMZN", "Amazon.com Inc.", "87.2", "Growing"),
                (5, "META", "Meta Platforms", "85.9", "Improving"),
                (6, "NVDA", "NVIDIA Corp.", "84.3", "Strong Momentum"),
                (7, "JPM", "JPMorgan Chase", "82.8", "Stable"),
                (8, "V", "Visa Inc.", "81.5", "Consistent"),
                (9, "MA", "Mastercard Inc.", "80.2", "Reliable"),
                (10, "TSLA", "Tesla Inc.", "79.8", "Volatile")
            ]
            
            # Add results to tree
            for result in dummy_results:
                tree.insert("", "end", values=result)
            
            # Reset progress
            progress.set(0)
        
        # Run scan in separate thread
        thread = threading.Thread(target=scan_thread)
        thread.start()

if __name__ == "__main__":
    app = SimpleScreenerApp()
    app.mainloop()

In [15]:
import customtkinter as ctk
from tkinter import ttk
import threading

class StockScreenerApp(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.title("Advanced Stock Screener")
        self.geometry("1200x800")
        
        # Configure theme
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
        
        # Define metrics for each category
        self.metrics = {
            "Fundamental": {
                "P/E Ratio": ["Low P/E (<10)", "Medium P/E (10-20)", "High P/E (>20)"],
                "Market Cap": ["Large Cap (>$10B)", "Mid Cap ($2-10B)", "Small Cap (<$2B)"],
                "Dividend Yield": ["High (>4%)", "Medium (2-4%)", "Low (<2%)", "No Dividend"],
                "Revenue Growth": ["High (>20%)", "Medium (10-20%)", "Low (<10%)"]
            },
            "Technical": {
                "RSI": ["Oversold (<30)", "Neutral (30-70)", "Overbought (>70)"],
                "Moving Averages": ["Above 200MA", "Between MAs", "Below 200MA"],
                "Volume": ["High Volume", "Average Volume", "Low Volume"],
                "Price Action": ["Uptrend", "Sideways", "Downtrend"]
            },
            "Sentiment": {
                "News Sentiment": ["Very Positive", "Positive", "Neutral", "Negative"],
                "Social Media": ["Trending Up", "Stable", "Trending Down"],
                "Analyst Ratings": ["Strong Buy", "Buy", "Hold", "Sell"]
            },
            "Overall": {
                "Combined Score": ["Excellent (90-100)", "Good (70-89)", "Fair (50-69)", "Poor (<50)"],
                "Risk Level": ["Low Risk", "Medium Risk", "High Risk"],
                "Growth Potential": ["High Growth", "Moderate Growth", "Low Growth"]
            }
        }
        
        self.create_gui()
        
    def create_gui(self):
        # Create notebook
        self.notebook = ttk.Notebook(self)
        self.notebook.pack(fill="both", expand=True, padx=10, pady=5)
        
        # Create tabs
        self.tabs = {}
        for category in self.metrics.keys():
            self.tabs[category] = self.create_tab(category)
    
    def create_tab(self, category):
        tab = ctk.CTkFrame(self.notebook)
        self.notebook.add(tab, text=category)
        
        # Top frame for metric selection
        top_frame = ctk.CTkFrame(tab)
        top_frame.pack(fill="x", padx=10, pady=5)
        
        # Metric selection
        ctk.CTkLabel(top_frame, text="Select Metric:").pack(side="left", padx=5)
        metric_var = ctk.StringVar(value=list(self.metrics[category].keys())[0])
        metric_dropdown = ctk.CTkComboBox(
            top_frame,
            values=list(self.metrics[category].keys()),
            variable=metric_var,
            command=lambda x: self.update_criteria(category, x)
        )
        metric_dropdown.pack(side="left", padx=5)
        
        # Criteria selection
        criteria_frame = ctk.CTkFrame(top_frame)
        criteria_frame.pack(side="left", fill="x", expand=True, padx=10)
        
        criteria_var = ctk.StringVar(value=self.metrics[category][metric_var.get()][0])
        criteria_dropdown = ctk.CTkComboBox(
            top_frame,
            values=self.metrics[category][metric_var.get()],
            variable=criteria_var
        )
        criteria_dropdown.pack(side="left", padx=5)
        
        # Scan button
        scan_button = ctk.CTkButton(
            top_frame,
            text="Find Top 10",
            command=lambda: self.run_scan(category)
        )
        scan_button.pack(side="right", padx=10)
        
        # Progress bar
        progress = ctk.CTkProgressBar(tab)
        progress.pack(fill="x", padx=10, pady=5)
        progress.set(0)
        
        # Results table
        columns = ("Rank", "Symbol", "Company", "Score", "Details")
        tree = ttk.Treeview(tab, columns=columns, show="headings", height=10)
        
        # Configure columns
        widths = {"Rank": 50, "Symbol": 100, "Company": 300, "Score": 100, "Details": 300}
        for col in columns:
            tree.heading(col, text=col)
            tree.column(col, width=widths.get(col, 150))
        
        tree.pack(fill="both", expand=True, padx=10, pady=5)
        
        # Scrollbar
        scrollbar = ttk.Scrollbar(tab, orient="vertical", command=tree.yview)
        scrollbar.pack(side="right", fill="y")
        tree.configure(yscrollcommand=scrollbar.set)
        
        return {
            "frame": tab,
            "metric": metric_var,
            "criteria": criteria_var,
            "progress": progress,
            "tree": tree
        }
    
    def update_criteria(self, category, metric):
        # Update criteria dropdown when metric changes
        tab = self.tabs[category]
        criteria_values = self.metrics[category][metric]
        tab["criteria"].set(criteria_values[0])
        
        # Update the criteria dropdown with new values
        for widget in tab["frame"].winfo_children():
            if isinstance(widget, ctk.CTkFrame):
                for child in widget.winfo_children():
                    if isinstance(child, ctk.CTkComboBox) and child.winfo_name() != metric:
                        child.configure(values=criteria_values)
                        break
    
    def run_scan(self, category):
        # Get tab components
        tab = self.tabs[category]
        tree = tab["tree"]
        progress = tab["progress"]
        selected_metric = tab["metric"].get()
        selected_criteria = tab["criteria"].get()
        
        # Clear existing results
        for item in tree.get_children():
            tree.delete(item)
        
        # Simulate scan with dummy data
        dummy_data = [
            (1, "AAPL", "Apple Inc.", "95", "Strong Match"),
            (2, "MSFT", "Microsoft Corp.", "92", "Very Good Match"),
            (3, "GOOGL", "Alphabet Inc.", "88", "Good Match")
        ]
        
        # Show progress
        for i in range(len(dummy_data)):
            progress.set((i + 1) / len(dummy_data))
            self.update_idletasks()
            import time
            time.sleep(0.2)
            tree.insert("", "end", values=dummy_data[i])
        
        # Reset progress
        progress.set(0)

if __name__ == "__main__":
    app = StockScreenerApp()
    app.mainloop()

# Web-ready

In [4]:
import streamlit as st
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go

# Configure page settings
st.set_page_config(
    page_title="AI Stock Screener",
    page_icon="📈",
    layout="wide"
)

# Define screening criteria
METRICS = {
    "Fundamental": {
        "P/E Ratio": ["Low (<10)", "Medium (10-20)", "High (>20)"],
        "Market Cap": ["Large (>$10B)", "Mid ($2-10B)", "Small (<$2B)"],
        "Dividend Yield": ["High (>4%)", "Medium (2-4%)", "Low (<2%)"],
        "Revenue Growth": ["High (>20%)", "Medium (10-20%)", "Low (<10%)"]
    },
    "Technical": {
        "RSI": ["Oversold (<30)", "Neutral (30-70)", "Overbought (>70)"],
        "Moving Averages": ["Above 200MA", "Between MAs", "Below 200MA"],
        "Volume": ["High", "Average", "Low"],
        "Price Action": ["Uptrend", "Sideways", "Downtrend"]
    },
    "Sentiment": {
        "News Sentiment": ["Very Positive", "Positive", "Neutral", "Negative"],
        "Social Media": ["Bullish", "Neutral", "Bearish"],
        "Analyst Ratings": ["Strong Buy", "Buy", "Hold", "Sell"]
    },
    "Overall": {
        "Combined Score": ["Excellent (90-100)", "Good (70-89)", "Fair (50-69)", "Poor (<50)"]
    }
}

def main():
    # Header
    st.title("📈 AI-Powered Stock Screener")
    st.markdown("---")

    # Sidebar filters
    with st.sidebar:
        st.header("Screening Criteria")
        
        category = st.selectbox(
            "Select Category",
            options=list(METRICS.keys())
        )
        
        metric = st.selectbox(
            "Select Metric",
            options=list(METRICS[category].keys())
        )
        
        criteria = st.selectbox(
            "Select Criteria",
            options=METRICS[category][metric]
        )
        
        st.markdown("---")
        scan_button = st.button("🔍 Find Top 10", use_container_width=True)

    # Main content area
    if scan_button:
        # Show progress
        progress_bar = st.progress(0)
        status_text = st.empty()
        
        # Simulate screening process
        for i in range(100):
            progress_bar.progress(i + 1)
            status_text.text(f"Scanning stocks... {i+1}%")
            
        # Display results in columns
        col1, col2 = st.columns([2, 1])
        
        with col1:
            st.subheader("Top 10 Matches")
            
            # Sample data (replace with real screening results)
            results_df = pd.DataFrame({
                "Rank": range(1, 11),
                "Symbol": ["AAPL", "MSFT", "GOOGL", "AMZN", "META", 
                          "NVDA", "TSM", "AVGO", "ASML", "AMD"],
                "Company": ["Apple Inc.", "Microsoft Corp.", "Alphabet Inc.", 
                          "Amazon.com Inc.", "Meta Platforms Inc.",
                          "NVIDIA Corp.", "Taiwan Semiconductor", 
                          "Broadcom Inc.", "ASML Holding", "Advanced Micro Devices"],
                "Score": [95, 92, 88, 85, 82, 80, 78, 75, 73, 70],
                "Trend": ["↑", "↑", "→", "↑", "↓", "↑", "→", "↑", "→", "↓"]
            })
            
            st.dataframe(
                results_df,
                column_config={
                    "Score": st.column_config.ProgressColumn(
                        "Score",
                        help="Screening score based on selected criteria",
                        format="%d%%",
                        min_value=0,
                        max_value=100,
                    ),
                },
                hide_index=True,
                use_container_width=True
            )

        with col2:
            st.subheader("Score Distribution")
            
            # Create score distribution chart
            fig = go.Figure(data=[go.Bar(
                x=results_df['Symbol'],
                y=results_df['Score'],
                marker_color='rgb(26, 118, 255)'
            )])
            
            fig.update_layout(
                xaxis_title="Symbol",
                yaxis_title="Score",
                height=400,
                margin=dict(l=0, r=0, t=0, b=0)
            )
            
            st.plotly_chart(fig, use_container_width=True)

        # Detailed analysis for top stock
        st.markdown("---")
        st.subheader("Top Stock Analysis")
        
        top_stock = results_df.iloc[0]
        
        metrics_col1, metrics_col2, metrics_col3 = st.columns(3)
        
        with metrics_col1:
            st.metric(
                "Current Score",
                f"{top_stock['Score']}%",
                "↑ 5%"
            )
            
        with metrics_col2:
            st.metric(
                "Rank Change",
                "1",
                "↑ 2 positions"
            )
            
        with metrics_col3:
            st.metric(
                "Trend",
                "Bullish",
                "↑ Strong"
            )

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'plotly'